In [19]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [20]:
df = pd.read_csv("../Data/processed-data/Senators_sentiment.csv")

We use a pre-trained RoBERTa model to classify emotions for each statement

In [21]:
# Initialize pipeline
model_name = "SamLowe/roberta-base-go_emotions"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

emotion_pipeline = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    padding=True,
    truncation=True,
    max_length=512
)

# Define the function for emotion detection
def detect_emotion(text):
    if isinstance(text, str) and len(text.strip()) > 0:
        try:
            result = emotion_pipeline(text)
            return result[0]['label'], result[0]['score']
        except IndexError:
            return None, None
    return None, None

df[['emotion', 'confidence']] = df['message'].apply(lambda x: pd.Series(detect_emotion(x)))


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


We use Latent Dirichlet Allocation (LDA) to identify dominant topics for each statement

In [22]:
text_data = df['message'].dropna().astype(str)

# Vectorize text data
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
text_vectors = vectorizer.fit_transform(text_data)

# Fit LDA model
num_topics = 15
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(text_vectors)

def get_top_words(model, feature_names, n_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics[topic_idx] = ', '.join(top_words)
    return topics

n_top_words = 10
feature_names = vectorizer.get_feature_names_out()
topics = get_top_words(lda, feature_names, n_top_words)

print("Discovered Topics:")
for topic_idx, words in topics.items():
    print(f"Topic {topic_idx}: {words}")

# Assign dominant topics
topic_probabilities = lda.transform(text_vectors)
topic_assignments = topic_probabilities.argmax(axis=1)
topic_labels = [topics[topic_idx] for topic_idx in topic_assignments]

filtered_df = df.loc[df['message'].notna()].copy()
filtered_df['dominant_topic'] = topic_labels
filtered_df['topic_probability'] = topic_probabilities.max(axis=1)

df = pd.concat([df, filtered_df[['dominant_topic', 'topic_probability']]], axis=1)

output_file = "../Data/processed-data/Senators_final.csv"
df.to_csv(output_file, index=False)

Discovered Topics:
Topic 0: climate, change, economy, energy, environment, planet, action, policy, protect, global
Topic 1: deal, new, green, biden, drive, cost, choice, bank, consumer, car
Topic 2: climate, change, act, energy, inflation, reduction, job, clean, economy, threat
Topic 3: climate, change, action, global, opportunity, wont, thats, president, energy, im
Topic 4: energy, climate, american, change, job, green, create, time, union, biden
Topic 5: climate, change, real, solution, gas, problem, make, funding, action, trump
Topic 6: climate, job, biden, change, american, today, policy, gas, emission, crisis
Topic 7: climate, change, need, agreement, community, infrastructure, protect, economy, im, address
Topic 8: climate, change, energy, crisis, impact, reduction, act, inflation, threat, american
Topic 9: climate, change, administration, biden, policy, energy, people, wont, care, farmer
Topic 10: climate, change, need, american, clean, energy, crisis, continue, country, tax
Top

Now let's do it for newly elected President Donald Trump

In [6]:
president_df = pd.read_csv("../Data/processed-data/President_sentiment.csv")

In [7]:
model_name = "SamLowe/roberta-base-go_emotions"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

emotion_pipeline = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    padding=True,          
    truncation=True,       
    max_length=512         
)

def detect_emotion(text):
    if isinstance(text, str) and len(text.strip()) > 0:
        try:
            result = emotion_pipeline(text)
            return result[0]['label'], result[0]['score']
        except IndexError:
            return None, None
    return None, None

president_df[['emotion', 'confidence']] = president_df['message'].apply(
    lambda x: pd.Series(detect_emotion(x))
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [8]:
text_data = president_df['message'].dropna().astype(str)

vectorizer = CountVectorizer(
    max_df=0.95,  
    min_df=2,    
    stop_words='english'  
)
text_vectors = vectorizer.fit_transform(text_data)

num_topics = 5 
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(text_vectors)

def get_top_words(model, feature_names, n_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics[topic_idx] = ', '.join(top_words)  
    return topics

n_top_words = 10
feature_names = vectorizer.get_feature_names_out()
topics = get_top_words(lda, feature_names, n_top_words)

print("Discovered Topics:")
for topic_idx, words in topics.items():
    print(f"Topic {topic_idx}: {words}")

topic_probabilities = lda.transform(text_vectors)  
topic_assignments = topic_probabilities.argmax(axis=1) 

topic_labels = [topics[topic_idx] for topic_idx in topic_assignments]

filtered_df = president_df.loc[president_df['message'].notna()].copy()
filtered_df['dominant_topic'] = topic_labels  
filtered_df['topic_probability'] = topic_probabilities.max(axis=1)

president_df = pd.concat([president_df, filtered_df[['dominant_topic', 'topic_probability']]], axis=1)

output_file = "../Data/processed-data/President_final.csv"
president_df.to_csv(output_file, index=False)

Discovered Topics:
Topic 0: climate, warming, global, change, air, going, clean, nuclear, country, obama
Topic 1: climate, change, warming, global, working, changed, cold, called, state, weather
Topic 2: climate, change, warming, global, work, anymore, changed, world, freezing, let
Topic 3: global, warming, ice, record, people, freezing, cold, snow, coldest, country
Topic 4: warming, global, cold, change, climate, hell, need, new, freezing, country


I now have two new CSV files saved. I will do analysis in another file because I will be switching to R.